In [22]:
import cv2
import numpy as np

# Create a window to display the video feed
cv2.namedWindow('Webcam')
cv2.setMouseCallback('Webcam', lambda event, x, y, flags, param: onMouse(event, x, y, flags, param))

# Initialize the polygon vertices list and current polygon list
polygons = []
current_polygon = []

def onMouse(event, x, y, flags, param):
    global current_polygon

    # When left mouse button is pressed, add vertex to current polygon
    if event == cv2.EVENT_LBUTTONDOWN:
        current_polygon.append((x,y))

    # When right mouse button is pressed, close current polygon and add it to the list
    elif event == cv2.EVENT_RBUTTONDOWN:
        if len(current_polygon) > 2:
            polygons.append(current_polygon)
            current_polygon = []

# Initialize the webcam capture object
cap = cv2.VideoCapture(0)

# Start capturing webcam input
while True:
    ret, frame = cap.read()

    # Draw all polygons on top of the video feed
    for polygon in polygons:
        pts = np.array(polygon, np.int32)
        pts = pts.reshape((-1,1,2))
        cv2.polylines(frame,[pts],True,(0,255,0))

    # Draw the current polygon on top of the video feed
    if len(current_polygon) > 0:
        pts = np.array(current_polygon, np.int32)
        pts = pts.reshape((-1,1,2))
        cv2.polylines(frame,[pts],True,(0,0,255))

    # Display the video feed and wait for a key press
    cv2.imshow('Webcam', frame)

    # Check for key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam
cap.release()

# Calculate the average HSV values of the pixels within each polygon
hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
for i, polygon in enumerate(polygons):
    pts = np.array(polygon, np.int32)
    pts = pts.reshape((-1,1,2))
    mask = np.zeros_like(frame[:,:,0])
    cv2.fillPoly(mask, [pts], 255)

    # Create a mask where the lines are black
    line_mask = np.zeros_like(frame[:,:,0])
    cv2.polylines(line_mask,[pts],True,(255,255,255))

    # Combine the polygon mask and line mask to create a new mask
    mask = cv2.bitwise_xor(mask, line_mask)

    # Calculate the average HSV values using the new mask
    avg_hsv = cv2.mean(hsv, mask=mask)
    print(f"Polygon {i}: {avg_hsv}")

# Close all windows
cv2.destroyAllWindows()

Polygon 0: (26.278978388998034, 128.01768172888015, 200.6738703339882, 0.0)
Polygon 1: (26.991596638655462, 126.05882352941175, 196.00420168067225, 0.0)
Polygon 2: (27.45918367346939, 140.13673469387757, 181.9530612244898, 0.0)


In [25]:
import numpy as np
import cv2

def get_limits(hsvC):
    hue = hsvC[0][0][0]  # Get the hue value

    # Handle red hue wrap-around
    if hue >= 165:  # Upper limit for divided red hue
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([180, 255, 255], dtype=np.uint8)
    elif hue <= 15:  # Lower limit for divided red hue
        lowerLimit = np.array([0, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    else:
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)

    return lowerLimit, upperLimit

def determinar_color(hsvC):
    # Convertir los valores de hsvC de [0, 360] a [0, 180] para la matiz
    hsvC[:, :, 0] = hsvC[:, :, 0] / 2
    hsvC = hsvC.astype(np.uint8)

    lowerLimit, upperLimit = get_limits(hsvC)
    color = "Desconocido"

    # Definir los rangos de color en HSV
    colores = {
        'naranja': (np.array([10, 100, 100]), np.array([20, 255, 255])),
        'amarillo': (np.array([20, 100, 100]), np.array([30, 255, 255])),
        'verde': (np.array([30, 100, 100]), np.array([60, 255, 255])),
        'azul': (np.array([90, 100, 100]), np.array([120, 255, 255])),
        'blanco': (np.array([0, 0, 150]), np.array([180, 50, 255])),
        'rojo': (lowerLimit, upperLimit)
    }

    for nombre_color, (limite_inferior, limite_superior) in colores.items():
        if cv2.inRange(hsvC, limite_inferior, limite_superior).any():
            color = nombre_color
            break

    return color

# Ejemplo de uso
if __name__ == "__main__":
    # Simular un valor de hsvC
    Polygon_0 = [31.278978388998034, 128.01768172888015, 200.6738703339882]  # Valores HSV
    hsvC = np.array([[Polygon_0]], dtype=np.float32)  # Convertir a matriz de numpy

    # Determinar el color predominante
    color_predominante = determinar_color(hsvC)

    print("El color predominante es:", color_predominante)




El color predominante es: naranja
